In [1]:
import os
import sys

if not "SimpleHTR/src" in sys.path:
    sys.path.append("SimpleHTR/src")
if not "WordSegmentation/src" in sys.path:
    sys.path.append("WordSegmentation/src")
if not "CTCWordBeamSearch/py" in sys.path:
    sys.path.append("CTCWordBeamSearch/py")

In [2]:
import matplotlib.pyplot as plt

import cv2
from WordSegmentation import wordSegmentation, prepareImg

import sys
import argparse
import cv2
import editdistance
from DataLoader import DataLoader, Batch
from Model import Model, DecoderType
from SamplePreprocessor import preprocess

import editdistance
from Metrics import Metrics

/home/system-gru/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
class FilePaths:
    "filenames and paths to data"
    fnCharList = 'SimpleHTR/model/charList.txt'
    fnAccuracy = 'SimpleHTR/model/accuracy.txt'
    fnTrain = 'SimpleHTR/data/'
    fnInfer = 'SimpleHTR/data/test.png'
    fnCorpus = 'SimpleHTR/data/corpus.txt'

In [7]:
filePath = "/home/system-gru/lines/"
f=open(filePath+"lines.txt")

fileNames = []
gtTexts = []

for (i, line) in enumerate(f):
    if not line or line[0]=='#':
        continue
    lineSplit = line.strip().split(' ')
    assert len(lineSplit) >= 9
    
    fileNameSplit = lineSplit[0].split('-')
    fileName = fileName = filePath + fileNameSplit[0] + '/' \
            + fileNameSplit[0] + '-' + fileNameSplit[1] + '/' + lineSplit[0] + '.png'
    
    gtText = lineSplit[8]
    if(len(lineSplit) > 9 ):
        for i in range(9, len(lineSplit)):
            gtText += " " + lineSplit[i] 
    gtArr = gtText.split('|')
    
    gtText = ""
    for text in gtArr:
        gtText += text + " "
    gtText = gtText.strip()
    
    fileNames.append(fileName)
    gtTexts.append(gtText)
    
del fileName, gtText

In [8]:
decoderType = DecoderType.WordBeamSearch
model = Model(open(FilePaths.fnCharList).read(), decoderType, mustRestore=True)

Instructions for updating:
dim is deprecated, use axis instead
Python: 3.6.5 |Anaconda custom (64-bit)| (default, Apr 29 2018, 16:14:56) 
[GCC 7.2.0]
Tensorflow: 1.13.0-dev20181028
Init with stored values from SimpleHTR/model/snapshot-38
INFO:tensorflow:Restoring parameters from SimpleHTR/model/snapshot-38


In [10]:
def line_to_words(img):
    img = prepareImg(img, 50)
    res = wordSegmentation(img, kernelSize=25, sigma=11, theta=7, minArea=100)
    
    return res

def word_to_text(img):
    processed_img = preprocess(img, Model.imgSize)
    batch = Batch(None, [processed_img])
    (recognized, probability) = model.inferBatch(batch, True)
    
    return (recognized, probability)

In [14]:
m = Metrics()

for (i,f) in enumerate(fileNames):
    try:
        img = cv2.imread(f)
        res = line_to_words(img)
    
        p_text = ""
        for(j, w) in enumerate(res):
            (wordBox, wordImg) = w
            (recognized, probability) = word_to_text(wordImg)
        
            p_text += recognized[0] + " "
    
        p_text = p_text.strip()
        m.addSample(p_text, gtTexts[i])
        
        if i % 10 == 0 and i != 0:
            print("Index:\t%d" %i)
            print("File name: %s" %fileNames[i])
        
            print("Predict:\t" + p_text)
            print("Ground truth:\t" + gtTexts[i])
    
            strEditDist = str(editdistance.eval(p_text, gtTexts[i]))
            print("Editdistance:\t" + strEditDist + '\n')
        
        if i % 100 == 0 and i != 0:
            print("CER rate: ", m.getCER())
            print("WER rate: ", m.getWER())
            print()
    except Exception as e:
        print("Error in: %s" %fileNames[i])
        print(e)

Index:	10
File name: /home/system-gru/lines/a01/a01-000x/a01-000x-03.png
Predict:	Food ha pur down ' resolution on the subject
Ground truth:	Foot has put down a resolution on the subject
Editdistance:	4

Index:	20
File name: /home/system-gru/lines/a01/a01-003/a01-003-07.png
Predict:	that the House of Lords should be abolished
Ground truth:	that the House of Lords should be abolished
Editdistance:	0

Index:	30
File name: /home/system-gru/lines/a01/a01-003u/a01-003u-06.png
Predict:	ato existence they should not now put
Ground truth:	into existence , they should not now put
Editdistance:	4

Index:	40
File name: /home/system-gru/lines/a01/a01-003x/a01-003x-05.png
Predict:	should not now put-forward nom nes He believes
Ground truth:	should not now put forward nominees . He believes
Editdistance:	5

Index:	50
File name: /home/system-gru/lines/a01/a01-007/a01-007-06.png
Predict:	Parties of Northern Rhodesia have agreed to
Ground truth:	Parties of Northern Rhodesia have agreed to
Editdistance:

Index:	420
File name: /home/system-gru/lines/a01/a01-077/a01-077-05.png
Predict:	whether America was lagging , behind
Ground truth:	whether America was lagging behind
Editdistance:	2

Index:	430
File name: /home/system-gru/lines/a01/a01-077u/a01-077u-05.png
Predict:	lagging behind Russia o missile power
Ground truth:	lagging behind Russia in missile power .
Editdistance:	4

Index:	440
File name: /home/system-gru/lines/a01/a01-082u/a01-082u-06.png
Predict:	od tsitsits best to avoid young Pressmen
Ground truth:	did his best to avoid giving Pressmen
Editdistance:	12

Index:	450
File name: /home/system-gru/lines/a01/a01-087/a01-087-06.png
Predict:	use behind the Health service the other
Ground truth:	issue behind the Health Service ; the other
Editdistance:	6

Index:	460
File name: /home/system-gru/lines/a01/a01-087u/a01-087u-07.png
Predict:	service the other tried to show that the
Ground truth:	Service ; the other tried to show that the
Editdistance:	3

Index:	470
File name: /home/system-

Index:	830
File name: /home/system-gru/lines/a02/a02-093/a02-093-04.png
Predict:	Commonwealth Relations secretary " for an nourishment ' talks
Ground truth:	Commonwealth Relations Secretary for an hour's talks
Editdistance:	12

Index:	840
File name: /home/system-gru/lines/a02/a02-098/a02-098-05.png
Predict:	from talks with Sir John Moffat Northern Rhodesian
Ground truth:	from talks with Sir John Moffat , Northern Rhodesian
Editdistance:	2

Index:	850
File name: /home/system-gru/lines/a02/a02-102/a02-102-06.png
Predict:	aged at i now studying history at Cambri-
Ground truth:	aged 24 , is now studying history at Cambri-
Editdistance:	5

Index:	860
File name: /home/system-gru/lines/a02/a02-106/a02-106-07.png
Predict:	rest of the country A Government report has
Ground truth:	rest of the country . A Government report has
Editdistance:	2

Index:	870
File name: /home/system-gru/lines/a02/a02-111/a02-111-07.png
Predict:	has been cancelled and secret plans prepared
Ground truth:	has been cancel

Index:	1230
File name: /home/system-gru/lines/a04/a04-050/a04-050-01.png
Predict:	not deny facilities to nations to-which 1wes
Ground truth:	not deny facilities to nations to which it was
Editdistance:	5

Index:	1240
File name: /home/system-gru/lines/a04/a04-054/a04-054-04.png
Predict:	purposes to be.under the se rather than on hand
Ground truth:	purposes to be under the sea rather than on land .
Editdistance:	5

Index:	1250
File name: /home/system-gru/lines/a04/a04-059/a04-059-07.png
Predict:	supports inte country
Ground truth:	supporters in the country .
Editdistance:	6

Index:	1260
File name: /home/system-gru/lines/a04/a04-069/a04-069-00.png
Predict:	To . ,n "million M Gaitskell said the
Ground truth:	To Mr. Macmillan , Mr Gaitskell said the
Editdistance:	11

Index:	1270
File name: /home/system-gru/lines/a04/a04-072/a04-072-01.png
Predict:	Union would b prepared to mock
Ground truth:	Union would be prepared to reach
Editdistance:	5

Index:	1280
File name: /home/system-gru/lines/a04/

Index:	1660
File name: /home/system-gru/lines/a06/a06-020/a06-020-04.png
Predict:	so this is an invitation from my good friends of the TUC that
Ground truth:	so this is an invitation from my good friends of the TUC that
Editdistance:	0

Index:	1670
File name: /home/system-gru/lines/a06/a06-025/a06-025-08.png
Predict:	co mm. Hess
Ground truth:	committees .
Editdistance:	7

Index:	1680
File name: /home/system-gru/lines/a06/a06-039/a06-039-01.png
Predict:	Union can be seen wre a truly cupping
Ground truth:	Union can be seen here - a truly inspiring
Editdistance:	9

Index:	1690
File name: /home/system-gru/lines/a06/a06-044/a06-044-00.png
Predict:	Mr. 1thou Fel Yarmouth called the decision shocking
Ground truth:	Mr. Anthony Fell ( Yarmouth ) called the decision " shocking "
Editdistance:	13

Index:	1700
File name: /home/system-gru/lines/a06/a06-051/a06-051-03.png
Predict:	darn a fenced minutes later Mr. Fell got up
Ground truth:	down . A few minutes later , Mr. Fell got up
Editdistance:	10


...

Index:	10770
File name: /home/system-gru/lines/m01/m01-026/m01-026-05.png
Predict:	for several-notes them had been-facing each-other across
Ground truth:	For several minutes they had been facing each other across
Editdistance:	8

Index:	10780
File name: /home/system-gru/lines/m01/m01-032/m01-032-07.png
Predict:	shell Af flesh and bone decad of intellect and
Ground truth:	shell of flesh and bone , devoid of intellect and
Editdistance:	6

Index:	10790
File name: /home/system-gru/lines/m01/m01-038/m01-038-08.png
Predict:	the empty minded receptiveness that
Ground truth:	the empty-minded receptiveness that
Editdistance:	1

Index:	10800
File name: /home/system-gru/lines/m01/m01-049/m01-049-08.png
Predict:	K I toes He stretched his as
Ground truth:	his toes . He stretched his arms
Editdistance:	7

CER rate:  0.15988895991065102
WER rate:  0.2501693724437307

Index:	10810
File name: /home/system-gru/lines/m01/m01-060/m01-060-06.png
Predict:	upward and shook his head slowly from
Ground truth:	

Index:	11150
File name: /home/system-gru/lines/m03/m03-114/m03-114-06.png
Predict:	so Mr Gilberto told us that held and
Ground truth:	so Mr Gilberto told us , that the wild and
Editdistance:	6

Index:	11160
File name: /home/system-gru/lines/m04/m04-000/m04-000-00.png
Predict:	NO0NE has .l satisfactorily explained how i
Ground truth:	NO ONE has ever satisfactorily explained how a
Editdistance:	7

Index:	11170
File name: /home/system-gru/lines/m04/m04-007/m04-007-02.png
Predict:	the bis the 220 from Dinas to Llangrwl
Ground truth:	the bus , the 2.20 from Dinas to Llangrwl ,
Editdistance:	6

Index:	11180
File name: /home/system-gru/lines/m04/m04-012/m04-012-02.png
Predict:	watched it go from the windows of the
Ground truth:	watched it go from the windows of the
Editdistance:	0

Index:	11190
File name: /home/system-gru/lines/m04/m04-019/m04-019-02.png
Predict:	the world is it Ifor- wr it 's " Du
Ground truth:	the world , is it ? " " For me it is , " Dai
Editdistance:	17

Index:	11200
File 

Index:	11560
File name: /home/system-gru/lines/n01/n01-052/n01-052-00.png
Predict:	They were neare Modena Geoffrey dreams
Ground truth:	They were nearing Modena . Geoffrey focused the radar scanner
Editdistance:	26

Index:	11570
File name: /home/system-gru/lines/n01/n01-052/n01-052-10.png
Predict:	refreshing hours and had now completed two-thirds
Ground truth:	refreshing hours and had now completed two-thirds
Editdistance:	0

Index:	11580
File name: /home/system-gru/lines/n01/n01-057/n01-057-07.png
Predict:	across Italy si Foggia to hole
Ground truth:	across Italy via Foggia to Barletta .
Editdistance:	10

Index:	11590
File name: /home/system-gru/lines/n02/n02-000/n02-000-07.png
Predict:	his flat for a drink
Ground truth:	his flat for a drink .
Editdistance:	2

Index:	11600
File name: /home/system-gru/lines/n02/n02-009/n02-009-00.png
Predict:	" - t caught mm gust as he was gang to the
Ground truth:	" I caught him just as he was going to the
Editdistance:	8

CER rate:  0.163986798850500

Index:	11970
File name: /home/system-gru/lines/n04/n04-075/n04-075-05.png
Predict:	beyond " "did san Bawley's name was
Ground truth:	to you ? " It did . Ian Bawley's name was
Editdistance:	12

Index:	11980
File name: /home/system-gru/lines/n04/n04-092/n04-092-00.png
Predict:	for the sake of accuracy " he said and , know
Ground truth:	" For the sake of accuracy , " he said , " and I
Editdistance:	13

Index:	11990
File name: /home/system-gru/lines/n04/n04-100/n04-100-01.png
Predict:	Cher and count-check potting but the truth
Ground truth:	Check and counter-check - nothing but the truth .
Editdistance:	10

Index:	12000
File name: /home/system-gru/lines/n04/n04-107/n04-107-03.png
Predict:	found him curiously stimulating Your a
Ground truth:	found him curiously stimulating . " You 're a
Editdistance:	7

CER rate:  0.16591772885042044
WER rate:  0.2553860776681452

Index:	12010
File name: /home/system-gru/lines/n04/n04-114/n04-114-04.png
Predict:	shook his head see paid for by His Lordship
G

KeyboardInterrupt: 

In [15]:
print("CER rate: ", m.getCER())
print("WER rate: ", m.getWER())
print()

CER rate:  0.16692975613134242
WER rate:  0.2556642863440007

